<a href="https://colab.research.google.com/github/ShoSato-047/DSCI330_Module_5_Web-Scraping/blob/main/DSCI330_act5_6_dealing_with_exceptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install composable

In [2]:
!pip install composable --upgrade

In [5]:
from composable.try_ import wrap_result, get_results
from composable.maybe import maybe, unmaybe
from composable.strict import map
from composable.object import obj

### Example Data

In [4]:
(L := ['a', 'B', 'c', '1', None])

['a', 'B', 'c', '1', None]

## Problem 1 - `NoneType` errors.

When trying to call a method on `None`, you get a `NoneType` error.

In [6]:
None.lower()

AttributeError: 'NoneType' object has no attribute 'lower'

In [7]:
[v.lower() for v in L]

AttributeError: 'NoneType' object has no attribute 'lower'

In [8]:
# Composable solution
 (L
 >> map(obj.lower())
)

IndentationError: unexpected indent (<ipython-input-8-6c5ec84adb1c>, line 2)

### Sources of `NoneType` errors.

Anything that can introduce `None` into your computation chain.

1. Python functions created with `def` without a `return` statement return `None`.
2. Non-matches from regular expressions defined with `re`.
3. Searching for non-existant attributes in `bs4`

In [12]:
# print does not have a return statement - "side effect"
def my_func(value):
    print(value + 1)

In [11]:
my_func(5) is None

6


True

In [13]:
my_func(5) + 1

6


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
# Other common examples
# tag.click()
# model.fit()

#### Functions with no return value

In [14]:
# Print doesn't have a return ==> returns None
[print(x).lower() for x in L]

a


AttributeError: 'NoneType' object has no attribute 'lower'

#### Non-matches in an `re`

In [18]:
import re

a_or_b = re.compile(r'^[ab]$')

[a_or_b.match(v).group(0) for v in L]

AttributeError: 'NoneType' object has no attribute 'group'

In [19]:
[a_or_b.match(v) if v is not None else None for v in L]

[<re.Match object; span=(0, 1), match='a'>, None, None, None, None]

In [20]:
from bs4 import BeautifulSoup

(example := BeautifulSoup('<div> Stuff </div'))

<html><body><div> Stuff </div></body></html>

In [21]:
example.find('div', class_ = lambda cls: 'Todd' in cls)

TypeError: argument of type 'NoneType' is not iterable

## Solution 1 - Use the 'Maybe' class to wrap optional data

In [22]:
from composable.maybe import Just, Nothing, Maybe

In [23]:
Maybe

composable.maybe.Nothing | composable.maybe.Just

In [25]:
# Just holds a string inside a box
Just('A').lower()

Just('a')

In [27]:
# replacement of None
Nothing().lower()

Nothing()

In [28]:
from composable.maybe import maybe

In [29]:
[maybe(v) for v in L]

[Just('a'), Just('B'), Just('c'), Just('1'), Nothing()]

In [35]:
from toolz import pipe

make_this_mine = lambda s: 'my_' + s

pipe(L,
     lambda L: [maybe(v) for v in L],
     lambda L: [v.lower() for v in L],
     lambda L: [v.map(make_this_mine) for v in L],
     lambda L: [unmaybe(v, default = "") for v in L]
    #  lambda L: [unmaybe(v) for v in L] # taking stirng out of the box
     )

['my_a', 'my_b', 'my_c', 'my_1', '']

In [37]:
(L
 >> map(maybe)
 >> map(obj.lower())
 >> map(obj.map(make_this_mine))
 >> map(unmaybe(default = ""))
)

['my_a', 'my_b', 'my_c', 'my_1', '']

## Problem 2 - Exceptions that halt computation and destroy your data

Source.

1. Bad code [name error],
2. Bad code [type error],
3. Bad code [value error],
3. Bad code [problems with the outside world - wifi, website locked, etc.].

In [38]:
[int(v) for v in L]

ValueError: invalid literal for int() with base 10: 'a'

## Solution 2 - Put the problem in the Try box

In [42]:
from composable.try_ import Try, Result, Error, wrap_result, get_results

In [40]:
Try

composable.try_.Error | composable.try_.Result

In [49]:
pipe(L,
     lambda L: [wrap_result(v) for v in L], # put everything in a box
     lambda L: [v.lower() for v in L],
    #  lambda L: [v.map(make_this_mine) for v in L] # make_this_mine() only works for string
     lambda L: [v.map(int) for v in L],
     lambda L: [get_results(v, error_handler=lambda err: err) for v in L]
     )

[Error(a, invalid literal for int() with base 10: 'a'),
 Error(b, invalid literal for int() with base 10: 'b'),
 Error(c, invalid literal for int() with base 10: 'c'),
 1,
 Error(None, 'NoneType' object has no attribute 'lower')]